# Componenten afhandelen

> tags: []

# Passing Data and Events Between Vaadin Components

Learn how to communicate between components in Vaadin Flow.

1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  Componenten afhandelen

-   [De geselecteerde contactpersoon in het formulier weergeven](https://vaadin.com/docs/latest/tutorial/handling-view-state/#showing-the-selected-contact-in-the-form)
-   [De lay-out responsief maken](https://vaadin.com/docs/latest/tutorial/handling-view-state/#making-the-layout-responsive)
-   [Formuliergebeurtenissen afhandelen](https://vaadin.com/docs/latest/tutorial/handling-view-state/#handling-form-events)

In het vorige hoofdstuk hebt u een herbruikbare formuliercomponent gemaakt om contactpersonen te bewerken. In dit hoofdstuk sluit u deze aan op de rest van de weergave en beheert u de weergavestatus. Het formulier:

-   toont het geselecteerde contact in het raster;
    
-   is verborgen wanneer er geen contact is geselecteerd;
    
-   slaat contactpersonen in de database op en verwijdert deze.
    

De eerste stap is om de geselecteerde rasterrij in het formulier weer te geven.

Werk hiervoor als volgt bij:`ListView`

```
package com.example.application.views.list;

import com.example.application.data.entity.Contact;
import com.example.application.data.service.CrmService;
import com.vaadin.flow.component.button.Button;
import com.vaadin.flow.component.grid.Grid;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.component.textfield.TextField;
import com.vaadin.flow.data.value.ValueChangeMode;
import com.vaadin.flow.router.PageTitle;
import com.vaadin.flow.router.Route;

@Route(value = "")
@PageTitle("Contacts | Vaadin CRM")
public class ListView extends VerticalLayout {
    Grid<Contact> grid = new Grid<>(Contact.class);
    TextField filterText = new TextField();
    ContactForm form;
    CrmService service;

    public ListView(CrmService service) {
        this.service = service;
        addClassName("list-view");
        setSizeFull();
        configureGrid();
        configureForm();

        add(getToolbar(), getContent());
        updateList();
        closeEditor(); 1
    }

    private HorizontalLayout getContent() {
        HorizontalLayout content = new HorizontalLayout(grid, form);
        content.setFlexGrow(2, grid);
        content.setFlexGrow(1, form);
        content.addClassNames("content");
        content.setSizeFull();
        return content;
    }

    private void configureForm() {
        form = new ContactForm(service.findAllCompanies(), service.findAllStatuses());
        form.setWidth("25em");
    }

    private void configureGrid() {
        grid.addClassNames("contact-grid");
        grid.setSizeFull();
        grid.setColumns("firstName", "lastName", "email");
        grid.addColumn(contact -> contact.getStatus().getName()).setHeader("Status");
        grid.addColumn(contact -> contact.getCompany().getName()).setHeader("Company");
        grid.getColumns().forEach(col -> col.setAutoWidth(true));

        grid.asSingleSelect().addValueChangeListener(event ->
            editContact(event.getValue())); 2
    }

    private HorizontalLayout getToolbar() {
        filterText.setPlaceholder("Filter by name...");
        filterText.setClearButtonVisible(true);
        filterText.setValueChangeMode(ValueChangeMode.LAZY);
        filterText.addValueChangeListener(e -> updateList());

        Button addContactButton = new Button("Add contact");
        addContactButton.addClickListener(click -> addContact()); 3

        HorizontalLayout toolbar = new HorizontalLayout(filterText, addContactButton);
        toolbar.addClassName("toolbar");
        return toolbar;
    }

    public void editContact(Contact contact) { 4
        if (contact == null) {
            closeEditor();
        } else {
            form.setContact(contact);
            form.setVisible(true);
            addClassName("editing");
        }
    }

    private void closeEditor() {
        form.setContact(null);
        form.setVisible(false);
        removeClassName("editing");
    }

    private void addContact() { 5
        grid.asSingleSelect().clear();
        editContact(new Contact());
    }


    private void updateList() {
        grid.setItems(service.findAllContacts(filterText.getValue()));
    }
}
```

1.  De call aan het einde van de constructeur:`closeEditor()`
    
    -   stelt de formuliercontactpersoon in op , oude waarden wissen;`null`
        
    -   verbergt het formulier;
        
    -   hiermee verwijdert u de CSS-klasse uit de weergave.`"editing"`
        
    
2.  `addValueChangeListener()` voegt een listener toe aan het raster. De component ondersteunt multi- en single-selectiemodi. U hoeft slechts één te selecteren, zodat u de methode kunt gebruiken. De methode retourneert de in de geselecteerde rij, of null als er geen selectie is.`Grid``Contact``asSingleSelect()``getValue()``Contact`
    
3.  Bel wanneer de gebruiker op de knop "Contact toevoegen" klikt.`addContact()`
    
4.  `editContact()` hiermee stelt u de geselecteerde contactpersoon in en verbergt of toont u het formulier, afhankelijk van de selectie. Het stelt ook de CSS-klassenaam in tijdens het bewerken.`ContactForm``"editing"`
    
5.  `addContact()` wist de rasterselectie en maakt een nieuw .`Contact`
    

Bouw de applicatie. U zou nu contacten in het raster moeten kunnen selecteren en ze in het formulier moeten kunnen zien. Maar geen van de knoppen werkt nog.

![Formulier met een contactpersoon](https://vaadin.com/docs/latest/static/2bac720abf51dbe414bd180819546383/03979/form-showing-contact.png)

## [](https://vaadin.com/docs/latest/tutorial/handling-view-state/#making-the-layout-responsive)De lay-out responsief maken

Om de lay-out responsief en bruikbaar te maken op zowel mobiel als desktop, moeten we CSS toevoegen.

Voeg hiervoor de volgende CSS toe aan frontend/themes/flowcrmtutorial/styles.css:

```
@media all and (max-width: 1100px) {
  .list-view.editing .toolbar,
  .list-view.editing .contact-grid {
    display: none;
  }
}
```

De CSS-mediaquery verbergt het raster en de werkbalk wanneer u contactpersonen op een smal scherm bewerkt.

## [](https://vaadin.com/docs/latest/tutorial/handling-view-state/#handling-form-events)Formuliergebeurtenissen afhandelen

De API is ontworpen om herbruikbaar te zijn; het is configureerbaar door eigenschappen en het vuurt de nodige gebeurtenissen. Tot nu toe hebt u een lijst met bedrijven, statussen en de contactpersoon doorgegeven aan het formulier. Nu hoeft u alleen maar naar de gebeurtenissen te luisteren om de integratie te voltooien.`ContactForm`

Als u gebeurtenislisteners wilt verwerken, moet u de methoden bijwerken en toevoegen.`configureForm()``saveContact()``deleteContact()`

```
private void configureForm() {
    form = new ContactForm(service.findAllCompanies(), service.findAllStatuses());
    form.setWidth("25em");
    form.addListener(ContactForm.SaveEvent.class, this::saveContact); 1
    form.addListener(ContactForm.DeleteEvent.class, this::deleteContact); 2
    form.addListener(ContactForm.CloseEvent.class, e -> closeEditor()); 3
}

private void saveContact(ContactForm.SaveEvent event) {
    service.saveContact(event.getContact());
    updateList();
    closeEditor();
}

private void deleteContact(ContactForm.DeleteEvent event) {
    service.deleteContact(event.getContact());
    updateList();
    closeEditor();
}
```

1.  De gebeurtenislistener save roept . Het:`saveContact()`
    
    1.  Hiermee wordt deze optie gebruikt om de contactpersoon in de gebeurtenis op te slaan in de database.`contactService`
        
    2.  Hiermee wordt de lijst bijgewerkt.
        
    3.  Hiermee sluit u de editor.
        
    
2.  De gebeurtenislistener delete roept aan . Het:`deleteContact()`
    
    1.  gebruikt om het contact uit de database te verwijderen;`contactService`
        
    2.  werkt de lijst bij;
        
    3.  sluit de editor.
        
    
3.  De close event listener sluit de editor.
    

Bouw de toepassing en controleer of u nu contactpersonen kunt selecteren, toevoegen, bijwerken en verwijderen.

![De geselecteerde contactpersoon heeft voor- en achternamen bijgewerkt.](https://vaadin.com/docs/latest/static/5d2e85656f36bce65ebd361dc206a105/03979/updated-contact.png)

7ADFAE2F-44BD-4EE2-A8E1-E8B49581856B

[TutorialFormulieren en validatie](https://vaadin.com/docs/latest/tutorial/forms-and-validation)[TutorialNavigatie en app-indeling](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts)

**Was deze pagina nuttig?**  
Laat hieronder een reactie of een vraag achter. Je kunt ook deelnemen aan de [chat op Discord](https://discord.gg/MYFq5RTbBn) of [vragen stellen op StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
